In [1]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

import buffalo.ingestion as ingestion
import buffalo.predictor as predictor
import buffalo.algorithm as algorithm
import buffalo.predictor.models as modeling
import torch
import pickle
import pandas as pd
from buffalo.utility import expand_grid
from tqdm.auto import tqdm

torch.manual_seed(0)

In [2]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [3]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [4]:
target_symbol = 'GE'

In [5]:
ingestor.data['ADJUSTED_DAILY_STOCK'].symbol.unique()

array(['AAPL', 'MSFT', 'IBM', 'META', 'JNJ', 'PFE', 'UNH', 'MARK', 'XLV',
       'JPM', 'BAC', 'GS', 'MS', 'XLF', 'PG', 'KO', 'PEP', 'NKE', 'XLP',
       'GE', 'HON', 'UTX', 'MMM', 'XLI'], dtype=object)

In [6]:
target_stock = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol == @target_symbol')[['open', 'high', 'low', 'adjusted_close', 'volume']]

In [7]:
other_stocks = ingestor.data['ADJUSTED_DAILY_STOCK'].query('symbol != @target_symbol')

In [9]:
for symbol in ['MSFT', 'IBM', 'JNJ', 'PFE', 'UNH', 'XLV', 'JPM', 'BAC', 'GS', 'XLF', 'AAPL', 'KO', 'PEP', 'NKE', 'XLP', 'PG', 'HON', 'MMM', 'XLI']:
    temp = other_stocks[other_stocks['symbol'] == symbol][['open', 'high', 'low', 'adjusted_close', 'volume']].rename(columns={'adjusted_close': 'close'})
    temp.columns = symbol + '_' + temp.columns
    target_stock = predictor.util.align_dataframe_by_time(target_stock, temp)

In [10]:
target_income_statement = ingestor.data['COMPANY_INCOME_STATEMENT'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_balance_sheet = ingestor.data['COMPANY_BALANCE_SHEET'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_cash_flow = ingestor.data['COMPANY_CASH_FLOW'].query('symbol == @target_symbol & freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function', 'net_income']).dropna(axis=1, how='all')
fed_funds_rate = ingestor.data['FEDERAL_FUNDS_RATE'][['value']].rename(columns={'value': 'effective_federal_funds_rate'}).dropna(axis=1, how='all')
payroll = ingestor.data['NONFARM_PAYROLL'][['value']].rename(columns={'value': 'total_nonfarm_payroll'}).dropna(axis=1, how='all')
cpi = ingestor.data['CPI'][['value']].rename(columns={'value': 'consumer_price_index'}).dropna(axis=1, how='all')
unemployment = ingestor.data['UNEMPLOYMENT'][['value']].rename(columns={'value': 'unemployment_rate'}).dropna(axis=1, how='all')
real_gdp = ingestor.data['REAL_GDP'][['value']].rename(columns={'value': 'real_gross_domestic_product'}).dropna(axis=1, how='all')
real_gdp_per_capita = ingestor.data['REAL_GDP_PER_CAPITA'][['value']].rename(columns={'value': 'real_gross_domestic_product_per_capita'})
treasury_yield = ingestor.data['TREASURY_YIELD'][['value', 'maturity']].pivot(columns=['maturity'], values=['value']).dropna(axis=1, how='all')
treasury_yield.columns = 'treasury_yield_' + treasury_yield.columns.droplevel(level=0)

In [11]:
sma = ingestor.data['SMA'].query('symbol == @target_symbol & interval == "daily"')
roc = ingestor.data['ROC'].query('symbol == @target_symbol & interval == "daily"')
ht_sine = ingestor.data['HT_SINE'].query('symbol == @target_symbol & interval == "daily"')
mom = ingestor.data['MOM'].query('symbol == @target_symbol & interval == "daily"')
sma = sma.pivot(columns=['time_period', 'series_type'], values=['sma']).dropna(axis=1, how='all')
sma.columns = sma.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, sma)
roc = roc.pivot(columns=['time_period', 'series_type'], values=['roc']).dropna(axis=1, how='all')
roc.columns = roc.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, roc)
ht_sine = ht_sine.pivot(columns=['time_period', 'series_type'], values=['lead_sine', 'sine']).dropna(axis=1, how='all')
ht_sine.columns = ht_sine.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, ht_sine)
mom = mom.pivot(columns=['time_period', 'series_type'], values=['mom']).dropna(axis=1, how='all')
mom.columns = mom.columns.map(lambda x: '-'.join([str(t) for t in x]))
target_stock = predictor.util.align_dataframe_by_time(target_stock, mom)

In [12]:
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, fed_funds_rate)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, payroll)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, cpi)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, unemployment)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, real_gdp_per_capita)
print(target_stock.shape)
target_stock = predictor.util.align_dataframe_by_time(target_stock, treasury_yield)
print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_income_statement)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_balance_sheet)
#print(target_stock.shape)
#target_stock = predictor.util.align_dataframe_by_time(target_stock, target_cash_flow)
#print(target_stock.shape)

(5692, 160)
(5691, 161)
(5671, 162)
(5671, 163)
(5671, 164)
(5286, 165)


c:\Users\carlo\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 83, using nperseg = 83
  warnings.warn('nperseg = {0:d} is greater than input length '


(5286, 166)
(5246, 171)


In [13]:
pickle.dump(target_stock, open(r'cached_data/target_stock.pickle', 'wb'))

In [14]:
target_stock = pickle.load(open(r'cached_data/target_stock.pickle', 'rb'))

In [15]:
target_stock.shape

(5246, 171)

In [16]:
n_head = 1

In [17]:
time_series_data = predictor.util.TimeSeriesData(endog=target_stock[['adjusted_close']], exog=target_stock.drop(columns=['adjusted_close']), seq_len=180, label_len=n_head, name=f'DAILY_ADJUSTED_CLOSE_{target_symbol}')

#### Offline Learning

In [20]:
sweep_params = expand_grid(
    hidden_size=[128, 256, 64, 32],
    num_layers=[1, 2, 4],
    dropout=[0.2, 0.4],
    batch_size=[64],
    learning_rate=[0.001],
    weight_decay=[0.001],
    epochs=[30, 60],
    bidirectional = [True, False],
    n_fold=1,
    activation='relu'
)

In [21]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.RNN(
        input_size=target_stock.shape[1],
        n_ahead=n_head,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    training_record = predictor.train_and_evaluate_model(
        rnn,
        optimizer,
        loss_func,
        time_series_data,
        epochs=param['epochs'],
        test_ratio=0.2,
        n_fold=param['n_fold'],
        clip_grad=1,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')

  0%|          | 0/96 [00:00<?, ?it/s]

c:\Users\carlo\miniconda3\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Multi-fold validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:223: RuntimeWarning: Mean of empty slice
  print(f'Averaged validation loss: {np.nanmean(train_valid_loss)}. Test loss: {test_loss}.')
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:236: RuntimeWarning: Mean of empty slice
  'average_validation_loss': np.nanmean(train_valid_loss) if non_empty_train_indices else None}


...

C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:223: RuntimeWarning: Mean of empty slice
  print(f'Averaged validation loss: {np.nanmean(train_valid_loss)}. Test loss: {test_loss}.')
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:236: RuntimeWarning: Mean of empty slice
  'average_validation_loss': np.nanmean(train_valid_loss) if non_empty_train_indices else None}
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:224: UserWarning: dataset_info with the same primary keys already exists with id 1, will not store dataset information.
  warn(f"dataset_info with the same primary keys already exists with id {searched_id}, will not store dataset information.")
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:397: UserWarning: model_info with the same primary keys already exists with id 47, will not store model information.
  warn(f'model_info with the same primary keys already exists with id {searched_id}, will not store model 

Multi-fold validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/60 [00:00<?, ?it/s]

Averaged validation loss: nan. Test loss: 1495.4912844447958.


C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:223: RuntimeWarning: Mean of empty slice
  print(f'Averaged validation loss: {np.nanmean(train_valid_loss)}. Test loss: {test_loss}.')
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:236: RuntimeWarning: Mean of empty slice
  'average_validation_loss': np.nanmean(train_valid_loss) if non_empty_train_indices else None}
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:224: UserWarning: dataset_info with the same primary keys already exists with id 1, will not store dataset information.
  warn(f"dataset_info with the same primary keys already exists with id {searched_id}, will not store dataset information.")
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:397: UserWarning: model_info with the same primary keys already exists with id 48, will not store model information.
  warn(f'model_info with the same primary keys already exists with id {searched_id}, will not store model 

In [22]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.LSTM(
        input_size=target_stock.shape[1],
        n_ahead=n_head,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    training_record = predictor.train_and_evaluate_model(
        rnn,
        optimizer,
        loss_func,
        time_series_data,
        epochs=param['epochs'],
        test_ratio=0.2,
        n_fold=param['n_fold'],
        clip_grad=1,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')

  0%|          | 0/96 [00:00<?, ?it/s]

c:\Users\carlo\miniconda3\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Multi-fold validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Averaged validation loss: nan. Test loss: 2188.6887725197435.


...

C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:223: RuntimeWarning: Mean of empty slice
  print(f'Averaged validation loss: {np.nanmean(train_valid_loss)}. Test loss: {test_loss}.')
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:236: RuntimeWarning: Mean of empty slice
  'average_validation_loss': np.nanmean(train_valid_loss) if non_empty_train_indices else None}
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:224: UserWarning: dataset_info with the same primary keys already exists with id 1, will not store dataset information.
  warn(f"dataset_info with the same primary keys already exists with id {searched_id}, will not store dataset information.")
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:397: UserWarning: model_info with the same primary keys already exists with id 95, will not store model information.
  warn(f'model_info with the same primary keys already exists with id {searched_id}, will not store model 

Multi-fold validation:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/60 [00:00<?, ?it/s]

Averaged validation loss: nan. Test loss: 1410.2486108326136.


C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:223: RuntimeWarning: Mean of empty slice
  print(f'Averaged validation loss: {np.nanmean(train_valid_loss)}. Test loss: {test_loss}.')
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\__init__.py:236: RuntimeWarning: Mean of empty slice
  'average_validation_loss': np.nanmean(train_valid_loss) if non_empty_train_indices else None}
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:224: UserWarning: dataset_info with the same primary keys already exists with id 1, will not store dataset information.
  warn(f"dataset_info with the same primary keys already exists with id {searched_id}, will not store dataset information.")
C:\Users\carlo\GitHub\ProjectBuffalo\buffalo\predictor\util.py:397: UserWarning: model_info with the same primary keys already exists with id 96, will not store model information.
  warn(f'model_info with the same primary keys already exists with id {searched_id}, will not store model 